# Head Files

In [1]:
import torch
import torchvision
# from torch.utils.tensorboard import SummaryWriter
import time
from torch import nn
from torch.utils.data import DataLoader
from load_data import MyData  # self-made
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm # View procedure
import os
import scipy.io
from random import random
import numpy as np
import gc
from torch.utils.tensorboard import SummaryWriter
from network_cnn_lstm_4 import MyNetwork
from torchnlp.word_to_vector import GloVe
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# 1. Prepare datasets

In [4]:
exper_dir = "conditionC"
# exper_dir = "rest"
hc_path = f"../data/eegmap_chunks/{exper_dir}/hc/{exper_dir}_hc.pt"
mcs_path = f"../data/eegmap_chunks/{exper_dir}/mcs/{exper_dir}_mcs.pt"
uws_path = f"../data/eegmap_chunks/{exper_dir}/uws/{exper_dir}_uws.pt"
hc = torch.load(hc_path)
mcs = torch.load(mcs_path)
uws = torch.load(uws_path)
print(hc.size())
print(mcs.size())
print(uws.size())

torch.Size([1231, 2400, 10, 11])
torch.Size([1353, 2400, 10, 11])
torch.Size([1220, 2400, 10, 11])


## 1.1 Split Map Data - Cross Fold Valid

In [8]:
# 定义函数用于保存数据集
def save_dataset(data, exper_dir, subset, mode, fold):
    torch.save(data, f"../data/eegmap_split/{exper_dir}/train/{subset}/{exper_dir}_{subset}_{mode}_{fold}.pt")

# test dataset
# 生成随机种子
torch.manual_seed(32)
# 生成随机索引
hc_indices = torch.randperm(len(hc))
mcs_indices = torch.randperm(len(mcs))
uws_indices = torch.randperm(len(uws))
test_percentage = 0.1
# hc
print(len(hc))
test_end = int(test_percentage * len(hc))
test_hc_indices = hc_indices[:test_end]
test_hc = hc[test_hc_indices]
torch.save(test_hc, f"../data/eegmap_split/{exper_dir}/test/hc/{exper_dir}_hc_test.pt")
hc = hc[hc_indices[test_end:]]
print(len(hc))
# mcs
print(len(mcs))
test_end = int(test_percentage * len(mcs))
test_mcs_indices = mcs_indices[:test_end]
test_mcs = mcs[test_mcs_indices]
torch.save(test_mcs, f"../data/eegmap_split/{exper_dir}/test/mcs/{exper_dir}_mcs_test.pt")
mcs = mcs[mcs_indices[test_end:]]
print(len(mcs))
#uws
print(len(uws))
test_end = int(test_percentage * len(uws))
test_uws_indices = uws_indices[:test_end]
test_uws = uws[test_uws_indices]
torch.save(test_uws, f"../data/eegmap_split/{exper_dir}/test/uws/{exper_dir}_uws_test.pt")
uws = uws[uws_indices[test_end:]]
print(len(uws))
# 清理内存
del hc_indices, mcs_indices, uws_indices, test_hc_indices, test_mcs_indices, test_uws_indices
del test_hc, test_mcs, test_uws
gc.collect()
torch.cuda.empty_cache()

# train dataset
# 生成随机索引
hc_indices = torch.randperm(len(hc))
mcs_indices = torch.randperm(len(mcs))
uws_indices = torch.randperm(len(uws))
fold_num = 5
# 划分数据集并保存
for fold in tqdm(range(fold_num)):
    # 划分 hc 数据集
    len_hc = len(hc)
    val_size = len_hc // fold_num  # 每一折的验证集大小
    val_start = fold * val_size  # 验证集起始索引
    val_end = min((fold + 1) * val_size, len_hc)  # 验证集结束索引
    train_hc_indices = torch.cat([hc_indices[:val_start], hc_indices[val_end:]])
    val_hc_indices = hc_indices[val_start:val_end]
    train_hc = hc[train_hc_indices]
    val_hc = hc[val_hc_indices]
    save_dataset(train_hc, exper_dir, 'hc', 'train', fold)
    save_dataset(val_hc, exper_dir, 'hc', 'val', fold)
    print(train_hc.size())
    print(val_hc.size())
    
    # 划分 mcs 数据集
    len_mcs = len(mcs)
    val_size = len_mcs // fold_num
    val_start = fold * val_size
    val_end = min((fold + 1) * val_size, len_mcs)
    train_mcs_indices = torch.cat([mcs_indices[:val_start], mcs_indices[val_end:]])
    val_mcs_indices = mcs_indices[val_start:val_end]
    train_mcs = mcs[train_mcs_indices]
    val_mcs = mcs[val_mcs_indices]
    save_dataset(train_mcs, exper_dir, 'mcs', 'train', fold)
    save_dataset(val_mcs, exper_dir, 'mcs', 'val', fold)
    print(train_mcs.size())
    print(val_mcs.size())
    
    # 划分 uws 数据集
    len_uws = len(uws)
    val_size = len_uws // fold_num
    val_start = fold * val_size
    val_end = min((fold + 1) * val_size, len_uws)
    train_uws_indices = torch.cat([uws_indices[:val_start], uws_indices[val_end:]])
    val_uws_indices = uws_indices[val_start:val_end]
    train_uws = uws[train_uws_indices]
    val_uws = uws[val_uws_indices]
    save_dataset(train_uws, exper_dir, 'uws', 'train', fold)
    save_dataset(val_uws, exper_dir, 'uws', 'val', fold)
    print(train_uws.size())
    print(val_uws.size())
    
    # 清理内存
    del train_hc_indices, train_mcs_indices, train_uws_indices,
    del val_hc_indices, val_mcs_indices, val_uws_indices
    del train_hc, train_mcs, train_uws, val_hc, val_mcs, val_uws
    gc.collect()
    torch.cuda.empty_cache()
    
# 清理内存
del hc_indices, mcs_indices, uws_indices
del hc, mcs, uws
gc.collect()
torch.cuda.empty_cache()

1231
1108
1353
1218
1220
1098


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:50: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([887, 2400, 10, 11])
torch.Size([221, 2400, 10, 11])
torch.Size([975, 2400, 10, 11])
torch.Size([243, 2400, 10, 11])
torch.Size([879, 2400, 10, 11])
torch.Size([219, 2400, 10, 11])
torch.Size([887, 2400, 10, 11])
torch.Size([221, 2400, 10, 11])
torch.Size([975, 2400, 10, 11])
torch.Size([243, 2400, 10, 11])
torch.Size([879, 2400, 10, 11])
torch.Size([219, 2400, 10, 11])
torch.Size([887, 2400, 10, 11])
torch.Size([221, 2400, 10, 11])
torch.Size([975, 2400, 10, 11])
torch.Size([243, 2400, 10, 11])
torch.Size([879, 2400, 10, 11])
torch.Size([219, 2400, 10, 11])
torch.Size([887, 2400, 10, 11])
torch.Size([221, 2400, 10, 11])
torch.Size([975, 2400, 10, 11])
torch.Size([243, 2400, 10, 11])
torch.Size([879, 2400, 10, 11])
torch.Size([219, 2400, 10, 11])
torch.Size([887, 2400, 10, 11])
torch.Size([221, 2400, 10, 11])
torch.Size([975, 2400, 10, 11])
torch.Size([243, 2400, 10, 11])
torch.Size([879, 2400, 10, 11])
torch.Size([219, 2400, 10, 11])


In [4]:
test_data = torch.load(f"../data/eegmap_split/{exper_dir}/test/hc/{exper_dir}_hc_test.pt")
print(test_data.size())

torch.Size([123, 2400, 10, 11])


## 1.2 Labels

In [10]:
# experimental dir: rest, conditionA, conditionB, conditionC
exper_dir = "conditionC"
# exper_dir = "rest"
root_dir = f"../data/eegmap_split/{exper_dir}"
# mode: train, test
mode = "train"
# condition_dir: hc, mcs, uws
condi_dir = "hc"
dataset = MyData(root_dir, mode, condi_dir) # eg.conditionA/hc
for person in tqdm(range(len(dataset))):
    filename = os.path.join(dataset.path, dataset.file_path[person])
    data_map = torch.load(filename)
    print(filename)
    print(data_map.size())
    name_without_extension = os.path.splitext(os.path.basename(filename))[0]
    label = torch.zeros(data_map.size(0)) # label = 0
    save_path = f"../data/eegmap_split/{exper_dir}/{mode}/{condi_dir}/{name_without_extension}_label.pt"
    torch.save(label, save_path)
    print(f"label_size:{label.size()}, label:{label[0]}")
    del label,data_map,filename
    gc.collect()
    torch.cuda.empty_cache()
condi_dir = "mcs"
dataset = MyData(root_dir, mode, condi_dir) # eg.conditionA/hc
for person in tqdm(range(len(dataset))):
    filename = os.path.join(dataset.path, dataset.file_path[person])
    data_map = torch.load(filename)
    print(filename)
    print(data_map.size())
    name_without_extension = os.path.splitext(os.path.basename(filename))[0]
    label = torch.ones(data_map.size(0)) # label = 1, for HC/DOC
#     label = torch.zeros(data_map.size(0)) # label = 0, for MCS/UWS
    save_path = f"../data/eegmap_split/{exper_dir}/{mode}/{condi_dir}/{name_without_extension}_label.pt"
    torch.save(label, save_path)
    print(f"label_size:{label.size()}, label:{label[0]}")
    del label,data_map,filename
    gc.collect()
    torch.cuda.empty_cache()
condi_dir = "uws"
dataset = MyData(root_dir, mode, condi_dir) # eg.conditionA/hc
for person in tqdm(range(len(dataset))):
    filename = os.path.join(dataset.path, dataset.file_path[person])
    data_map = torch.load(filename)
    print(filename)
    print(data_map.size())
    name_without_extension = os.path.splitext(os.path.basename(filename))[0]
#     label = torch.ones(data_map.size(0)) * 2 # label = 2
    label = torch.ones(data_map.size(0)) # label = 1
    save_path = f"../data/eegmap_split/{exper_dir}/{mode}/{condi_dir}/{name_without_extension}_label.pt"
    torch.save(label, save_path)
    print(f"label_size:{label.size()}, label:{label[0]}")
    del label,data_map,filename
    gc.collect()
    torch.cuda.empty_cache()

D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/1 [00:00<?, ?it/s]

../data/eegmap_split/conditionC\test\hc\conditionC_hc_test.pt
torch.Size([123, 2400, 10, 11])
label_size:torch.Size([123]), label:0.0


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1 [00:00<?, ?it/s]

../data/eegmap_split/conditionC\test\mcs\conditionC_mcs_test.pt
torch.Size([135, 2400, 10, 11])
label_size:torch.Size([135]), label:1.0


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1 [00:00<?, ?it/s]

../data/eegmap_split/conditionC\test\uws\conditionC_uws_test.pt
torch.Size([122, 2400, 10, 11])
label_size:torch.Size([122]), label:1.0


# 2. Train the model

## Hyperparameters and Related parameters

In [11]:
BATCH_SIZE = 1
C,H,W = 1,1,2400
learn_rate = 0.0005
num_epochs = 80

## Ensuring deterministicity through Random seeding

In [12]:
import random
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
manualSeed = 4
random.seed(manualSeed)
torch.manual_seed(manualSeed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(manualSeed)

## Setting the optimizer and Loss function

In [13]:
import torch.optim as optim
device = torch.device("cuda:0")
gc.collect()
torch.cuda.empty_cache()

# 定义LSTM超参数
input_size = 64  # 输入特征维度
hidden_size = 64  # 隐藏单元数量
num_layers = 2  # LSTM层数
output_size = 2  # 输出类别数量
model = MyNetwork(input_size, hidden_size, num_layers, output_size)
# # model.load_state_dict(torch.load("../model/Trial2.pt")) # !!!!!!!!!!!!!!!!!!!!!
model = model.to(device)

# ==损失函数权重
# ======== 二分类HC/DOC
# 计算总样本数量
# condition1
# total_samples = 887 + 985 + 879
# condition2
# total_samples = 929 + 1029 + 886
# condition3
total_samples = 887 + 975 + 879
# 计算每个类别的权重
# condition1
# weights = [total_samples / 887, total_samples / (985 + 879)]
# condition2
# weights = [total_samples / 929, total_samples / (1029 + 886)]
# condition3
weights = [total_samples / 887, total_samples / (975 + 879)]

# ======== 二分类MCS/UWS
# 计算总样本数量
# condition1
# total_samples = 985 + 879
# condition2
# total_samples = 1029 + 886
# condition3
# total_samples = 975 + 879
# 计算每个类别的权重
# condition1
# weights = [total_samples / 985, total_samples / 879]
# condition2
# weights = [total_samples / 1029, total_samples / 886]
# condition3
# weights = [total_samples / 975, total_samples / 879]

# # ======== 三分类
# # 计算总样本数量
# # conditionB
# total_samples = 929 + 1029 + 886
# # 计算每个类别的权重
# # conditionB
# weights = [total_samples / 929, total_samples / 1029, total_samples / 886]

# 将权重转换为张量
weights_tensor = torch.tensor(weights, device=device)

# 定义交叉熵损失函数并设置权重
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
# criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [7]:
# print(model)

## Training

In [ ]:
# experimental dir: rest, conditionA, conditionB, conditionC
# exper_dir = "rest"
exper_dir = "conditionC"
root_dir = f"../data/eegmap_split/{exper_dir}"
classification = "mcs_uws"
fold_num = 1
for fold in tqdm(range(fold_num)):
    # train num folds
    fold = 4
    # -- prepare datasets
    train_data = []
    train_label = []
    test_data = []
    test_label = []
#     # ---- hc
#     dataset = MyData(root_dir, "train", "hc") # hc
#     # find the fold file
#     count = 0
#     for person in range(len(dataset)):
#         filename = os.path.join(dataset.path, dataset.file_path[person])
#         # extract the pure name of the file
#         parts = filename.split("\\")
#         file_name = parts[-1]
#         name_without_extension = file_name.split(".")[0]
#         # label or data
#         file_last = name_without_extension.split("_")[-1]
#         if file_last.isdigit(): # data
#             # is this fold or not
#             if int(file_last) == fold: # yes
#                 print(filename)
#                 count = count + 1
#                 data_map = torch.load(filename)
#                 # train or valid
#                 if name_without_extension.split("_")[-2] == "train":
#                     for i in range(data_map.size(0)):
#                         train_data.append(data_map[i])
#                 elif name_without_extension.split("_")[-2] == "val":
#                     for i in range(data_map.size(0)):
#                         test_data.append(data_map[i])
#                 if count == 4:
#                     del data_map
#                     gc.collect()
#                     torch.cuda.empty_cache() 
#                     break
#             else:   # not
#                 pass
#         else: # label
#             # is this fold or not
#             file_last = name_without_extension.split("_")[-2]
#             if int(file_last) == fold: # yes
#                 print(filename)
#                 count = count + 1
#                 data_map = torch.load(filename)
#                 # train or valid
#                 if name_without_extension.split("_")[-3] == "train":
#                     for i in range(data_map.size(0)):
#                         train_label.append(data_map[i])
#                 elif name_without_extension.split("_")[-3] == "val":
#                     for i in range(data_map.size(0)):
#                         test_label.append(data_map[i])
#                 if count == 4:
#                     del data_map
#                     gc.collect()
#                     torch.cuda.empty_cache() 
#                     break
#             else:   # not
#                 pass
#         del filename, parts, file_name, name_without_extension, file_last
#         gc.collect()
#         torch.cuda.empty_cache()    
    # ---- mcs
    dataset = MyData(root_dir, "train", "mcs") # mcs
    # find the fold file
    count = 0
    for person in range(len(dataset)):
        filename = os.path.join(dataset.path, dataset.file_path[person])
        # extract the pure name of the file
        parts = filename.split("\\")
        file_name = parts[-1]
        name_without_extension = file_name.split(".")[0]
        # label or data
        file_last = name_without_extension.split("_")[-1]
        if file_last.isdigit(): # data
            # is this fold or not
            if int(file_last) == fold: # yes
                print(filename)
                count = count + 1
                data_map = torch.load(filename)
                # train or valid
                if name_without_extension.split("_")[-2] == "train":
                    for i in range(data_map.size(0)):
                        train_data.append(data_map[i])
                elif name_without_extension.split("_")[-2] == "val":
                    for i in range(data_map.size(0)):
                        test_data.append(data_map[i])
                if count == 4:
                    del data_map
                    gc.collect()
                    torch.cuda.empty_cache() 
                    break
            else:   # not
                pass
        else: # label
            # is this fold or not
            file_last = name_without_extension.split("_")[-2]
            if int(file_last) == fold: # yes
                print(filename)
                count = count + 1
                data_map = torch.load(filename)
                # train or valid
                if name_without_extension.split("_")[-3] == "train":
                    for i in range(data_map.size(0)):
                        train_label.append(data_map[i])
                elif name_without_extension.split("_")[-3] == "val":
                    for i in range(data_map.size(0)):
                        test_label.append(data_map[i])
                if count == 4:
                    del data_map
                    gc.collect()
                    torch.cuda.empty_cache() 
                    break
            else:   # not
                pass
        del filename, parts, file_name, name_without_extension, file_last
        gc.collect()
        torch.cuda.empty_cache()    
    # ---- uws
    dataset = MyData(root_dir, "train", "uws") # uws
    # find the fold file
    count = 0
    for person in range(len(dataset)):
        filename = os.path.join(dataset.path, dataset.file_path[person])
        # extract the pure name of the file
        parts = filename.split("\\")
        file_name = parts[-1]
        name_without_extension = file_name.split(".")[0]
        # label or data
        file_last = name_without_extension.split("_")[-1]
        if file_last.isdigit(): # data
            # is this fold or not
            if int(file_last) == fold: # yes
                print(filename)
                count = count + 1
                data_map = torch.load(filename)
                # train or valid
                if name_without_extension.split("_")[-2] == "train":
                    for i in range(data_map.size(0)):
                        train_data.append(data_map[i])
                elif name_without_extension.split("_")[-2] == "val":
                    for i in range(data_map.size(0)):
                        test_data.append(data_map[i])
                if count == 4:
                    del data_map
                    gc.collect()
                    torch.cuda.empty_cache() 
                    break
            else:   # not
                pass
        else: # label
            # is this fold or not
            file_last = name_without_extension.split("_")[-2]
            if int(file_last) == fold: # yes
                print(filename)
                count = count + 1
                data_map = torch.load(filename)
                # train or valid
                if name_without_extension.split("_")[-3] == "train":
                    for i in range(data_map.size(0)):
                        train_label.append(data_map[i])
                elif name_without_extension.split("_")[-3] == "val":
                    for i in range(data_map.size(0)):
                        test_label.append(data_map[i])
                if count == 4:
                    del data_map
                    gc.collect()
                    torch.cuda.empty_cache() 
                    break
            else:   # not
                pass 
        del filename, parts, file_name, name_without_extension, file_last
        gc.collect()
        torch.cuda.empty_cache()    
    print(torch.stack(train_data).size())
    print(torch.stack(train_label).size())
    print(torch.stack(test_data).size())
    print(torch.stack(test_label).size())
    del dataset
    gc.collect()
    torch.cuda.empty_cache()  
    
    train_data = torch.stack(train_data)
    train_label = torch.stack(train_label)
    test_data = torch.stack(test_data)
    test_label = torch.stack(test_label)
    # train dataset
    train_td = TensorDataset(train_data, train_label)
    train_loader = DataLoader(train_td, batch_size = BATCH_SIZE, shuffle = True)
    # test dataset
    test_td = TensorDataset(test_data, test_label)
    test_loader = DataLoader(test_td, batch_size = BATCH_SIZE, shuffle = True)
    del train_data, train_label, test_data, test_label, train_td, test_td
    gc.collect()
    torch.cuda.empty_cache()
    
    # set mode for each fold
    model = MyNetwork(input_size, hidden_size, num_layers, output_size)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr = learn_rate)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40, 60], gamma=0.2)
    # -- start training
    start_time = time.time()
    # train and test step records
    total_train_step = 0
    total_test_step = 0
    min_test_loss = 1000
    # add Tensorboard
    writer_train = SummaryWriter(f"../logs/{classification}/{exper_dir}_CNN_spa_lstm/logs_train_{fold}")
    writer_valid = SummaryWriter(f"../logs/{classification}/{exper_dir}_CNN_spa_lstm/logs_valid_{fold}")
    writer_valid_acc = SummaryWriter(f"../logs/{classification}/{exper_dir}_CNN_spa_lstm/logs_valid_acc_{fold}")
    for i in tqdm(range(num_epochs)):  
        print(f"========= Epoch {i} Training =========")
        # train steps
        model.train()
        for data in train_loader:
            # x, y
            data_map, label=data
            data_map_reshaped = torch.reshape(data_map, (110, 1, 1, 2400))
            label_int = label.long()
            data_map_reshaped=data_map_reshaped.to(device)
            label_int=label_int.to(device)
            del data_map, label
            gc.collect()
            torch.cuda.empty_cache()
            # y_pred
            label_pred = model(data_map_reshaped)
            # Loss Computation and Optimization
            loss = criterion(label_pred,label_int)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # draw tensorboard
            total_train_step = total_train_step + 1
            # print info
            if total_train_step % 1000 == 0:
                end_time = time.time()
                print(label_pred)
                print(f"Train time: {end_time - start_time}")
                print(f"Train steps: {total_train_step}, Loss: {loss.item()}")
            writer_train.add_scalar("train_loss",loss.item(),total_train_step)
            # Clear gpu
            del data, data_map_reshaped, label_int, label_pred, loss
            gc.collect()
            torch.cuda.empty_cache()
        
        # Evaluation and save the best model
        print(f"========= Epoch {i} Testing =========")
        model.eval()
        total_test_loss = 0
        test_count = 0
        total_test_acc = 0
        with torch.no_grad():
            for data in test_loader:
                test_count = test_count + 1
                # x, y
                data_map, label=data
                data_map_reshaped = torch.reshape(data_map, (110, 1, 1, 2400))
                label_int = label.long()
                data_map_reshaped = data_map_reshaped.to(device)
                label_int = label_int.to(device)
                del data_map, label
                gc.collect()
                torch.cuda.empty_cache()
                # y_pred
                label_pred_test = model(data_map_reshaped)
                loss = criterion(label_pred_test,label_int)
#                 print(label_pred_test)
                # accuracy 
                total_test_acc = total_test_acc + ((label_pred_test.argmax(1)) == label_int).sum()
                # draw tensorboad
                total_test_loss = total_test_loss + loss
                if test_count % 100 == 0:
                    print(f"Loss: {total_test_loss} Accuracy: {total_test_acc/test_count}")
                # Clear gpu
                del data_map_reshaped, label_int, label_pred_test, loss, data
                gc.collect()
                torch.cuda.empty_cache()
        print(f"Total Loss: {total_test_loss} Total Accuracy: {total_test_acc/test_count}")
        writer_valid.add_scalar("test_loss", total_test_loss, total_test_step)
        writer_valid_acc.add_scalar("test_acc", total_test_acc/test_count, total_test_step)
        total_test_step = total_test_step + 1
        print("..........Saving the model..........")
        torch.save(model.state_dict(),f"../model/{classification}/{exper_dir}_CNN_spa_lstm/Fold{fold}_Epoch{i}.pt") 
#         if total_test_loss < min_test_loss:
#             min_test_loss = total_test_loss
#             print("..........Saving the model..........")
#             torch.save(model.state_dict(),f"../model/{exper_dir}/Fold{fold}_Epoch{i}.pt")

D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/1 [00:00<?, ?it/s]

../data/eegmap_split/conditionC\train\mcs\conditionC_mcs_train_4.pt
../data/eegmap_split/conditionC\train\mcs\conditionC_mcs_train_4_label.pt
../data/eegmap_split/conditionC\train\mcs\conditionC_mcs_val_4.pt
../data/eegmap_split/conditionC\train\mcs\conditionC_mcs_val_4_label.pt
../data/eegmap_split/conditionC\train\uws\conditionC_uws_train_4.pt
../data/eegmap_split/conditionC\train\uws\conditionC_uws_train_4_label.pt
../data/eegmap_split/conditionC\train\uws\conditionC_uws_val_4.pt
../data/eegmap_split/conditionC\train\uws\conditionC_uws_val_4_label.pt
torch.Size([1854, 2400, 10, 11])
torch.Size([1854])
torch.Size([462, 2400, 10, 11])
torch.Size([462])


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:220: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/80 [00:00<?, ?it/s]

========= Epoch 0 Training =========
tensor([[0.4090, 0.4429]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 138.31700015068054
Train steps: 1000, Loss: 0.7102173566818237
========= Epoch 0 Testing =========
Loss: 71.78729248046875 Accuracy: 0.5099999904632568
Loss: 141.45748901367188 Accuracy: 0.5199999809265137
Loss: 212.9304656982422 Accuracy: 0.5200000405311584
Loss: 282.4778137207031 Accuracy: 0.5149999856948853
Total Loss: 328.2361145019531 Total Accuracy: 0.5151515007019043
..........Saving the model..........
========= Epoch 1 Training =========
tensor([[-1.6192, -0.7912]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 320.9880225658417
Train steps: 2000, Loss: 0.36251598596572876
tensor([[-0.4642, -0.2575]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 458.7530641555786
Train steps: 3000, Loss: 0.8018275499343872
========= Epoch 1 Testing =========
Loss: 68.99330139160156 Accuracy: 0.5899999737739563
Loss: 144.15415954589844 Accuracy: 0.56999999284

Loss: 29.11870765686035 Accuracy: 0.8899999856948853
Loss: 53.650081634521484 Accuracy: 0.8949999809265137
Loss: 92.91840362548828 Accuracy: 0.8866667151451111
Loss: 123.23455810546875 Accuracy: 0.8899999856948853
Total Loss: 134.20249938964844 Total Accuracy: 0.8917748928070068
..........Saving the model..........
========= Epoch 13 Training =========
tensor([[ 5.7406, 17.6428]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 3803.937484025955
Train steps: 25000, Loss: 6.794905857532285e-06
========= Epoch 13 Testing =========
Loss: 19.691431045532227 Accuracy: 0.9399999976158142
Loss: 40.01441955566406 Accuracy: 0.9350000023841858
Loss: 51.35788345336914 Accuracy: 0.9433333277702332
Loss: 78.27337646484375 Accuracy: 0.9325000047683716
Total Loss: 92.85108947753906 Total Accuracy: 0.9307359457015991
..........Saving the model..........
========= Epoch 14 Training =========
tensor([[-0.5986, 19.4012]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 3968.608506679535
Tra

In [ ]:
del train_data, train_label, test_data, test_label
gc.collect()
torch.cuda.empty_cache() 

In [8]:
test = torch.tensor([1])
print(test.size())
test = torch.tensor([test.item(),test.item()])
print(test.size())
print(test)

torch.Size([1])
torch.Size([2])
tensor([1, 1])


In [47]:
# Clear gpu
del optimizer
del scheduler
del criterion
del model
del data_map_pred
del label
del label_pred
del label_pred_test
del loss
del data

NameError: name 'data_map_pred' is not defined

In [49]:
gc.collect()
torch.cuda.empty_cache()